# Import packages

In [ ]:
# # if using google drive
# %cd /content/drive/MyDrive/ff_repo/Multifirefly-Project

In [ ]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

%load_ext autoreload
%autoreload 2


from data_wrangling import specific_utils, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_class, cluster_replacement_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_collect_info_class, GUAT_combine_info_class, process_GUAT_trials_class
from decision_making_analysis.compare_GUAT_and_TAFT import GUAT_vs_TAFT_class, GUAT_vs_TAFT_x_sessions_class, helper_GUAT_vs_TAFT_class
from visualization.matplotlib_tools import plot_trials, plot_behaviors_utils
from visualization.animation import animation_class
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curvature_class, curv_of_traj_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.plotly_polar_tools import plotly_utils_polar, plotly_for_ff_polar, plotly_for_trajectory_polar
from machine_learning.ml_methods import ml_methods_class

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import os, sys, sys
from importlib import reload
from sklearn.exceptions import ConvergenceWarning


plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 50



# Predict curvature

## explanatory_var

In [ ]:
reload(decision_making_utils)
reload(pattern_by_trials)
reload(decision_making_class)
reload(intended_targets_classes)
reload(find_best_arc)
reload(curvature_class)
reload(curvature_utils)


curv_temp = curvature_class.CurvatureOfPath(ff_dataframe, monkey_information, ff_caught_T_new, ff_real_position_sorted)
curv_temp.make_curvature_df([-1, 1])


ff_dataframe_temp = ff_dataframe.copy()
# eliminate ff that has abs(ff_angle) greater than 45 degrees
ff_dataframe_temp = ff_dataframe_temp[abs(ff_dataframe_temp['ff_angle_boundary']) <= math.pi/4]
ff_dataframe_truncated = ff_dataframe_temp[ff_dataframe_temp['time_since_last_vis'] <= 2.5]

moit = intended_targets_classes.ModelOfIntendedTargets(ff_dataframe_truncated, ff_caught_T_new, ff_real_position_sorted, monkey_information, ff_flash_sorted, ff_life_sorted,
                                                       time_range_of_trajectory=[-0.5, 2.5], num_time_points_for_trajectory=10)

# moit.get_and_process_manual_anno_long(n_seconds_after_crossing_boundary=2.5)

moit.manual_anno = pseudo_manual_anno
moit.manual_anno_long = pseudo_manual_anno_long
moit.eliminate_crossing_boundary_cases(n_seconds_after_crossing_boundary=0.5)
moit.get_input_data(num_ff_per_row=5, keeping_1_out_of_n_rows=10, 
                    add_arc_info=True, curvature_df=curv_temp.curvature_df, curv_of_traj_df=curv_of_traj_df.copy()
                    )

explanatory_var = moit.free_selection_x.copy()

## dependent_var

In [ ]:
# get dependent variables
reload(curvature_utils)
all_point_index = moit.chosen_rows_of_df.starting_point_index.values
curv_of_traj_df, traj_curv_descr = curv_of_traj_utils.find_curv_of_traj_df_based_on_curv_of_traj_mode(window_for_curv_of_traj, monkey_information, ff_caught_T_new, curv_of_traj_mode=curv_of_traj_mode, truncate_curv_of_traj_by_time_of_capture=truncate_curv_of_traj_by_time_of_capture)
dependent_var = curv_of_traj_df.curv_of_traj.values

## correlation in curv

In [ ]:
# This is kind of like for sanity check
old_curv = moit.free_selection_x_df.curv_of_traj.values.copy()
new_curv = curv_of_traj_df.curv_of_traj.values.copy()
stats.pearsonr(old_curv, new_curv)

## train test split

In [ ]:
keep_whole_chunk = False
test_size = 0.2

scaler = StandardScaler()
dependent_var_sc = scaler.fit_transform(dependent_var.reshape(-1,1)) #scale data
explanatory_var_sc = scaler.fit_transform(explanatory_var)


if keep_whole_chunk:
    indices = np.arange(len(dependent_var_sc))
    num_test_points = int(len(indices)*test_size)
    num_train_points = len(indices)-num_test_points
    # make sure that the test chunk will be a whole segment in the data, to minimize the splitting up of the train and test chunks
    test_indice_start = np.random.randint(0, num_train_points)
    indices_test = indices[test_indice_start:test_indice_start+num_test_points]
    indices_train = np.setdiff1d(indices, indices_test)
    X_train = explanatory_var_sc[indices_train]
    X_test = explanatory_var_sc[indices_test]
    y_train = dependent_var_sc[indices_train]
    y_test = dependent_var_sc[indices_test]
else:
    X_train, X_test, y_train, y_test = train_test_split(explanatory_var_sc, dependent_var_sc, test_size=test_size, random_state=42)

## ML

In [ ]:
# use MLPRegressor
mlp = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
mlp.score(X_test, y_test)

In [ ]:
# use KNeighborsRegressor
n_neighbors = int(math.sqrt(len(y_train)))
neigh = KNeighborsRegressor(n_neighbors=n_neighbors)
neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)

In [ ]:
# use RandomForestRegressor
regr = RandomForestRegressor(max_depth=10, random_state=0)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

In [ ]:
# use linear regression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)

# Hyperparameter tuning

### moit
Here, we tune parameters such as whether to pass in null arc info

In [ ]:
# make a dictionary of all the ff attributes combinations
ff_attributes=['ff_distance', 'ff_angle', 'time_since_last_vis']
ff_attributes_combinations = []
for i in range(1, len(ff_attributes)+1):
    combo_w_i_elements = list(itertools.combinations(ff_attributes, i))
    ff_attributes_combinations.extend([list(combo) for combo in combo_w_i_elements])
    
grid = {'ff_attributes': ff_attributes_combinations,
        'trajectory_data_kind': ['position', 'velocity', None],
        'add_arc_info': [True, False],
        'add_current_curv_of_traj': [True, False],
        'keep_whole_chunks' : [True, False],
        'num_ff_per_row': [5, 6, 7],  # Note: here's an inherent limitation to this: data where the target is not within the n_ff_per_row ff are eliminated. So data can be biased.
        'time_range_of_trajectory': [[-0.9, 0], [-0.9, 0.9]]
        }

# sample all the combinations from the grid
keys, values = zip(*grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

In [ ]:
reload(pattern_by_trials)
reload(decision_making_class)
reload(decision_making_utils)
reload(show_null_trajectory)
reload(intended_targets_classes)
reload(regression_utils)

combo = combinations[0]
result_df = pd.DataFrame()
for i in range(len(combinations)):
    combo = combinations[i].copy()
    print(i, 'out of', len(combinations))
    print(combo)
    combo['n_seconds_after_crossing_boundary'] = abs(combo['time_range_of_trajectory'][0])
    combo['n_seconds_before_crossing_boundary'] = abs(combo['time_range_of_trajectory'][1])
    # hide all printed material for below
    with general_utils.HiddenPrints():
        # hide all ConvergenceWarning
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=ConvergenceWarning)
            model_comparison_df = regression_utils.test_moit_hyperparameters(ff_dataframe, ff_caught_T_new, ff_real_position_sorted, monkey_information, ff_flash_sorted, ff_life_sorted, pseudo_manual_anno, pseudo_manual_anno_long,
                                curvature_df=curvature_df, keeping_1_out_of_n_rows=10, **combo)

    combo_for_df = combo.copy()
    combo_for_df['ff_attributes'] = [combo_for_df['ff_attributes']]
    combo_for_df['time_range_of_trajectory'] = [combo_for_df['time_range_of_trajectory']]
    # store the most accurate model
    combo_for_df['best_model'] = model_comparison_df.iloc[0]['model']
    # store the accuracy of the most accurate model
    combo_for_df['best_model_accuracy'] = round(model_comparison_df.iloc[0]['accuracy'], 3)
    # store the accuracies of all the other models
    for index, row in model_comparison_df.iterrows():
        combo_for_df[row['model']] = round(row['accuracy'], 3)
    temp_result_df = pd.DataFrame(combo_for_df)
    if i == 0:
        result_df = temp_result_df
    else:
        result_df = pd.concat([result_df, temp_result_df], axis=0)
    

In [ ]:
result_df

In [ ]:
stop!

### dm_replacement

In [ ]:
# make a dictionary of all the ff attributes combinations
ff_attributes=['ff_distance', 'ff_angle', 'time_since_last_vis']
ff_attributes_combinations = []
for i in range(1, len(ff_attributes)+1):
    combo_w_i_elements = list(itertools.combinations(ff_attributes, i))
    ff_attributes_combinations.extend([list(combo) for combo in combo_w_i_elements])
    
grid = {'ff_attributes': ff_attributes_combinations,
        'trajectory_data_kind': ['position', 'velocity', None],
        'add_arc_info': [True, False],
        'add_current_curv_of_traj': [True, False],
        'time_range_of_trajectory': [[-0.9, 0], [-0.9, 0.9]],
        'num_time_points_for_trajectory': [5, 10, 15],
        'replacement_inputs_format': ['diff_between_old_and_new', 'old_plus_diff_between_old_and_new', 'both_old_and_new'],
        }

# sample all the combinations from the grid
keys, values = zip(*grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

In [ ]:
reload(pattern_by_trials)
reload(decision_making_class)
reload(decision_making_utils)
reload(show_null_trajectory)
reload(intended_targets_classes)
reload(regression_utils)

combo = combinations[0]
dm_result_df = pd.DataFrame()
for i in range(len(combinations)):
    combo = combinations[i].copy()
    print(i, 'out of', len(combinations))
    print(combo)
    combo['n_seconds_after_crossing_boundary'] = abs(combo['time_range_of_trajectory'][0])
    combo['n_seconds_before_crossing_boundary'] = abs(combo['time_range_of_trajectory'][1])
    # hide all printed material for below
    with general_utils.HiddenPrints():
        # hide all ConvergenceWarning
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=ConvergenceWarning)
            model_comparison_df = regression_utils.test_dm_replacement_hyperparameters(ff_dataframe, ff_caught_T_new, ff_real_position_sorted, monkey_information, pseudo_manual_anno, curvature_df=curvature_df, **combo)

    combo_for_df = combo.copy()
    combo_for_df['ff_attributes'] = [combo_for_df['ff_attributes']]
    combo_for_df['time_range_of_trajectory'] = [combo_for_df['time_range_of_trajectory']]
    # store the most accurate model
    combo_for_df['best_model'] = model_comparison_df.iloc[0]['model']
    # store the accuracy of the most accurate model
    combo_for_df['best_model_accuracy'] = round(model_comparison_df.iloc[0]['accuracy'], 3)
    # store the accuracies of all the other models
    for index, row in model_comparison_df.iterrows():
        combo_for_df[row['model']] = round(row['accuracy'], 3)
    temp_result_df = pd.DataFrame(combo_for_df)
    if i == 0:
        dm_result_df = temp_result_df
    else:
        dm_result_df = pd.concat([dm_result_df, temp_result_df], axis=0)
    

In [ ]:

# def test_dm_replacement_hyperparameters(ff_dataframe, ff_caught_T_new, ff_real_position_sorted, monkey_information, pseudo_manual_anno,
#                               add_arc_info=True, add_current_curv_of_traj=True, furnish_with_trajectory_data=True, num_time_points_for_trajectory=20,
#                               ff_attributes=['ff_distance', 'ff_angle', 'time_since_last_vis'], trajectory_data_kind=['position'], curvature_df=None,
#                               time_range_of_trajectory=[-0.8, 0.8], n_seconds_before_crossing_boundary=0.8, n_seconds_after_crossing_boundary=0.8,
#                               replacement_inputs_format = 'diff_between_old_and_new'):

#     dm = decision_making_class.DecisionMaking(ff_dataframe, ff_caught_T_new, ff_real_position_sorted, monkey_information,
#                                             time_range_of_trajectory=time_range_of_trajectory, num_time_points_for_trajectory=num_time_points_for_trajectory)
#     dm.manual_anno = pseudo_manual_anno
#     dm.separate_manual_anno()
#     dm.eliminate_crossing_boundary_cases(n_seconds_before_crossing_boundary=n_seconds_before_crossing_boundary, n_seconds_after_crossing_boundary=n_seconds_after_crossing_boundary) 
#     dm.get_replacement_x_df(add_arc_info=add_arc_info, add_current_curv_of_traj=add_current_curv_of_traj, curvature_df=curvature_df, ff_attributes=ff_attributes, replacement_inputs_format=replacement_inputs_format)
#     dm.prepare_data_for_machine_learning(kind="replacement", furnish_with_trajectory_data=furnish_with_trajectory_data, trajectory_data_kind=trajectory_data_kind) 
#     dm.split_data_to_train_and_test(scaling_data=True)
#     dm.use_machine_learning_model(model=None)  

#     return dm.model_comparison_df

### Gaussian NB

var_smooth: 1e-09

In [ ]:
X_train = moit.X_train
y_train = moit.y_train

In [ ]:
# reload(regression_utils)
# reload(hyperparam_tuning_class)
# gnb = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
# grid = {'var_smoothing': np.logspace(0,-9, num=100)}
# gnb.random_search(grid=grid, model=GaussianNB(), n_iter=50, n_folds=3, n_repeats=1)


# grid = {'var_smoothing': np.logspace(-2,-9, num=100)}
# gnb.grid_search(grid=grid, model=GaussianNB(), n_folds=3, n_repeats=1)        

### Adaptive boosting

'n_estimators': 500,
'learning_rate': 0.05

In [ ]:
# reload(hyperparam_tuning_class)
# adap_boosting = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
# grid = {'n_estimators': [10, 50, 100, 500],
#         'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0]}
# adap_boosting.random_search(grid=grid, model=AdaBoostClassifier(), n_iter=50, n_folds=3, n_repeats=1)


adap_boosting = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
grid = {'n_estimators': [10, 50, 100, 200, 300, 500],
        'learning_rate': [0.05, 0.1, 0.2, 0.4, 0.7, 1.0]}
adap_boosting.grid_search(grid=grid, model=AdaBoostClassifier(), n_folds=3, n_repeats=1)        

### Gradient Boosting
learning_rate=0.05, max_depth=7, n_estimators=500, subsample=0.5


In [ ]:
# Might also need to train on: max_features='sqrt', bootstrap=True, min_samples_split=7, min_samples_leaf=2

In [ ]:
# reload(hyperparam_tuning_class)
# gradient_boosting = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
# grid = {'n_estimators': [10, 50, 100, 200, 300, 400, 500], 
#         'learning_rate': [0.01, 0.05, 0.1], 
#         'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 
#         'max_depth': [3, 5, 7, 9] + [int(x) for x in np.linspace(20, 50, num = 4)]}
# gradient_boosting.random_search(grid=grid, model=GradientBoostingClassifier(), n_iter=50, n_folds=3, n_repeats=1)

In [ ]:
gradient_boosting = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
grid = {'n_estimators': [10, 50, 100, 200, 300, 500], 
        'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1], 
        'subsample': [0.5, 0.7, 1.0], 
        'max_depth': [3, 7, 9]}
gradient_boosting.grid_search(grid=grid, model=GradientBoostingClassifier(), n_folds=3, n_repeats=1) 

### Bagging

n_estimators: 200
max_features: 0.9
bootstrap_features: True
bootstrap: True

In [ ]:
# reload(hyperparam_tuning_class)
# bagging = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
# grid = {'n_estimators': [10, 20, 50, 100, 200, 300, 500, 800],
#         'max_features': [0.90, 0.92, 0.95, 1.0],
#         'bootstrap': [True, False],
#         'bootstrap_features': [True, False]} 
# bagging.random_search(grid=grid, model=BaggingClassifier(), n_iter=50, n_folds=3, n_repeats=1)


bagging = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
grid = {'n_estimators': [50, 100, 200, 300, 400, 500],
        'max_features': [0.6, 0.65, 0.7, 0.75, 0.8, 0.85]}
bagging.grid_search(grid=grid, model=BaggingClassifier(bootstrap=True, bootstrap_features=True), n_folds=3, n_repeats=1)        

### Random forest

bootstrap: True

In [ ]:
# reload(hyperparam_tuning_class)
# rf = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
# grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)],
#         'max_features': [None, 'sqrt'],
#         'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)] + [None],
#         'min_samples_split': [2, 5, 10],
#         'min_samples_leaf': [1, 2, 4],
#         'bootstrap': [True, False]}

# rf.random_search(grid=grid, model=RandomForestRegressor(), n_iter=50, n_folds=3, n_repeats=1)

In [ ]:
y_train.dtype

In [ ]:
rf = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 600, num = 5)],
        'max_features': [None, 'sqrt'],
        'max_depth': [80, 90, 100],
        'min_samples_split': [2, 5, 7],
        'min_samples_leaf': [1, 2, 3]}
rf.grid_search(grid=grid, model=RandomForestRegressor(bootstrap=True), n_folds=3, n_repeats=1)        

### Logistic regression

In [ ]:
reload(hyperparam_tuning_class)
lr = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
grid = {'solver': ['newton-cg', 'lbfgs', 'liblinear'],
        'penalty': ['l2'],
        'C': [100, 10, 1.0, 0.1, 0.01]}

lr.random_search(grid=grid, model=LogisticRegression(), n_iter=50, n_folds=3, n_repeats=1)

In [ ]:
# grid = {'solvers': ['newton-cg', 'lbfgs', 'liblinear'],
#         'penalty': ['l2'],
#         'C': [100, 10, 1.0, 0.1, 0.01]}
# lr.grid_search(grid=grid, model=LogisticRegression(), n_folds=3, n_repeats=1)        

### SVM

In [ ]:
reload(hyperparam_tuning_class)
lr = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
grid = {'kernel': ['poly', 'rbf', 'sigmoid'],
        'C': [50, 10, 1.0, 0.1, 0.01],
        'gamma': ['scale']}

lr.random_search(grid=grid, model=SVC(), n_iter=50, n_folds=3, n_repeats=1)

In [ ]:
# grid = {'kernel': ['poly', 'rbf', 'sigmoid'],
#         'C': [50, 10, 1.0, 0.1, 0.01],
#         'gamma': ['scale']}
# lr.grid_search(grid=grid, model=SVC(), n_folds=3, n_repeats=1)        

### MLP

In [ ]:
reload(hyperparam_tuning_class)
mlp = hyperparam_tuning_class.HyperparameterTuning(X_train=X_train, y_train=y_train)
grid = {
    'hidden_layer_sizes': [(10,30,10),(20,), (50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'max_iter': [50, 100, 150],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

mlp.random_search(grid=grid, model=MLPClassifier(), n_iter=50, n_folds=3, n_repeats=1)

In [ ]:
# grid = {
#     'hidden_layer_sizes': [(10,30,10),(20,), (50,50,50), (50,100,50), (100,)],
#     'activation': ['tanh', 'relu'],
#     'max_iter': [50, 100, 150],
#     'solver': ['sgd', 'adam'],
#     'alpha': [0.0001, 0.05],
#     'learning_rate': ['constant','adaptive'],
# }
# mlp.grid_search(grid=grid, model=SVC(), n_folds=3, n_repeats=1)        

# Animation

## basic

In [ ]:
stop!

In [ ]:
additional_plotting_kwargs = {'show_eye_positions': False,
                     'show_eye_positions_on_the_right': False,
                     'show_connect_path_eye_positions': False,
                     'show_ff_indices': True,
}

temp_animation_plot_kwargs = data_item.animation_plot_kwargs.copy()
temp_animation_plot_kwargs['images_dir'] = None

for key, value in additional_plotting_kwargs.items():
    temp_animation_plot_kwargs[key] = value
temp_animation_plot_kwargs['show_connect_path_ff'] = False
temp_animation_plot_kwargs['show_connect_path_ff_memory'] = False
temp_animation_plot_kwargs['show_stops'] = False

temp_animation_plot_kwargs['trail_color_var'] = "grey"
plt.rcParams["figure.figsize"] = (10, 10)

In [ ]:
# change the default figure size
additional_anim_kwargs = {'plot_eye_position': True}
plt.rcParams["figure.figsize"] = (12, 12)

for i in range(1150, 1510, 30):
    #duration = [i, i+5]
    duration = [i, i+35]
    print(duration)
    video_dir = "/Users/dusiyi/Documents/Multifirefly-Project/for_manual_annotation2"
    file_name = f"time_{duration[0]}_to_{duration[1]}.mp4"
    data_item.make_animation(duration=duration, 
                             save_video=True, video_dir=video_dir, file_name=file_name, 
                             show_ff_indices=True, 
                            static_plot_on_the_left=False, 
                            animation_plot_kwargs=temp_animation_plot_kwargs, 
                            plot_time_index=True, 
                            show_speed_through_path_color=True,
                            max_num_frames=None, max_duration=None, min_duration=1, set_xy_limits=False,
                            **additional_anim_kwargs
                            )
    HTML(data_item.anim.to_html5_video()) 


In [ ]:
# delete all plt.rcParams['animation.ffmpeg_path'] =

## convert manual_anno

convert manual_anno based on the points deleted

In [ ]:
# for each row in manual_anno
manual_anno['new_point_index'] = np.nan
for index, row in manual_anno.iterrows():
    points_to_subtract = len(np.where(out_points < row['starting_point_index'])[0])
    manual_anno.at[index, 'new_point_index'] = row['starting_point_index'] - points_to_subtract
# see how many points are in out_points before this point, then subtract that from the number of points in this row

In [ ]:
manual_anno['new_point_index'] = manual_anno['new_point_index'].astype(int)

In [ ]:
manual_anno.head(20)

In [ ]:
dm = decision_making_class.DecisionMaking(ff_dataframe, ff_caught_T_new, ff_real_position_sorted, monkey_information,
                                          time_range_of_trajectory=[-1.5, 1.5], num_time_points_for_trajectory=20)

#dm.retrieve_manual_anno()
dm.retrieve_manual_anno()
manual_anno = dm.manual_anno.copy()